In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def tpm(A,v0,k,eps= 1e-6,maxiter=50000):
    # set t iterations to 0
    t = 0
    # for first iteration, set current vector to be initial
    v_c = v0
    # define q to be length - k
    q = len(v0) - k
    for i in range(maxiter):
        # find Ax/norm(Ax)
        v_cp = np.dot(A,v_c)/np.linalg.norm(np.dot(A,v_c))
        # find inidices of q lowest values
        F_idx = np.sort(np.argpartition(v_cp,q)[:q])
        # set q lowest entries to 0
        v_cp[F_idx] = 0
        # normalize after truncating
        v_cp = v_cp/np.linalg.norm(v_cp)
        #print(v_cp)
        t +=1
        # termination criterion
        if np.abs(max(np.abs(v_c)) - max(np.abs(v_cp))) < eps:
            break
        v_c = v_cp
    return t

## Final Code run for averaging out simulation convergence rate

In [3]:
kbar_t = []
klarge_t = []
c0_t = []
cclose_t = []
gapsmall_t = []
gaplarge_t = []
for q in range(1000):
    np.random.seed(2022+q)
    v1_l = np.random.uniform(0,1,4)
    v2_l = np.random.uniform(0,1,4)
    v1 = np.array([v1_l[0]]*5+[0]*480+[v1_l[1]]*5+[v1_l[2]]*5+[v1_l[3]]*5)
    v1 = v1/np.linalg.norm(v1)
    v2 = np.array([v2_l[0]]*5+[v2_l[1]]*5+[0]*480+[v2_l[2]]*5+[v2_l[3]]*5)
    v2 = v2/np.linalg.norm(v2)
    evals = np.array([200]+[100]+[50]*2+[10]+[5]+[1]*494)
    kbar = 1000 - sum(v1 == 0)
    # assign matrix where first 2 columns are dominant eigenvecs
    mat = np.zeros((500,500))
    mat[:,0] = v1
    mat[:,1] = v2
    # assign rest of eigenvectos to be randomly drawn from U(0,1)
    for i in range(498):
        mat[:,i+2] = np.random.uniform(0,1,size=500)
    V,nn = np.linalg.qr(mat)
    C = np.diag(evals)
    sigma = V@C@V.T
    for i in range(500):
        for j in range(500):
            if sigma[i,j] < 1e-10:
                sigma[i,j] = 0
    c0 = np.array([1]+[0]*499)
    # eigenvalue gap simulation
    evec_l = np.array([500]+[100]+[1]*498)
    C_l = np.diag(evec_l)
    evec_s = np.array([500]+[490]+[1]*498)
    C_s = np.diag(evec_s)
    sigma_l = V@C_l@V.T
    sigma_s = V@C_s@V.T
    gapsmall_t = np.append(gapsmall_t,tpm(sigma_s,c0,eps = 1e-8,k=kbar))
    gaplarge_t = np.append(gaplarge_t,tpm(sigma_l,c0,eps = 1e-8,k=kbar))
    
    # simulation for kbar vs large k
    
    kbar_t = np.append(kbar_t,tpm(sigma,c0,eps=1e-8,k=kbar))
    klarge_t = np.append(klarge_t,tpm(sigma,c0,eps=1e-8,k=400))
    
    # simulation for good initial
    cls_val = v1-0.001
    c0_t = np.append(c0_t,tpm(sigma,c0,eps=1e-8,k=kbar))
    cclose_t = np.append(cclose_t,tpm(sigma,cls_val,eps=1e-8,k=kbar))

## Calculating Metrics for Simulations

In [4]:
avg_kbar_t = np.sum(kbar_t)/1000
avg_klarge_t = np.sum(klarge_t)/1000
avg_c0_t = np.sum(c0_t)/1000
avg_cclose_t = np.sum(cclose_t)/1000
avg_gapsmall_t = np.sum(gapsmall_t)/1000
avg_gaplarge_t = np.sum(gaplarge_t)/1000

avg_pct_k = np.sum((klarge_t/kbar_t))/1000
avg_pct_c = np.sum((c0_t/cclose_t))/1000
avg_pct_g = np.sum((gapsmall_t/gaplarge_t))/1000

In [5]:
avg_kbar_t

15.962

In [6]:
avg_klarge_t

19.49

In [7]:
avg_pct_k

1.333942553893251

In [8]:
avg_c0_t

15.962

In [9]:
avg_cclose_t

11.335

In [10]:
avg_pct_c

1.4582388945935385

In [11]:
avg_gapsmall_t

100.123

In [12]:
avg_gaplarge_t

9.158

In [13]:
avg_pct_g

10.614318065268067